<a href="https://colab.research.google.com/github/jcmeve/MP/blob/main/MP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from IPython.core.display import Math
import numpy as np
import torch
from torch.utils.data import Dataset
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
from torch import nn
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import random
import json

In [2]:
from google.colab import drive
import pandas as pd
drive.mount('/content/drive')
filename = '/content/drive/MyDrive/Colab Notebooks/total_score.csv'
data = pd.read_csv(filename)
data = data.sample(frac=1).reset_index(drop=True) 
data.head()




Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


,documents__category_group_code,documents__category_group_name,documents__category_name,documents__distance,time,tag,base_score,쇼핑보너스,관광보너스,맛집보너스,score(0~10)
0,AD5,숙박,여행 > 숙박 > 호텔,3841,점심,맛집,0,0,0,0,0
1,MT1,대형마트,"가정,생활 > 대형마트",1293,오전,관광,2,2,2,2,2
2,FD6,음식점,"음식점 > 한식 > 해물,생선",4223,아침,맛집,1,1,1,4,4
3,FD6,음식점,음식점 > 한식 > 냉면,1865,오후,쇼핑,3,3,3,3,3
4,MT1,대형마트,"가정,생활 > 대형마트",2446,오후,쇼핑,4,7,4,4,7


In [3]:
from sklearn.model_selection import train_test_split

X = data.drop('score(0~10)', axis=1)
y = data['score(0~10)']
  
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.1)
print("X_train's shape : ", X_train.shape)
print("y_train's shape : ", y_train.shape)
print("\nX_test's shape : ", X_test.shape)
print("y_test's shape : ", y_test.shape)




X_train's shape :  (4374, 10)
y_train's shape :  (4374,)

X_test's shape :  (486, 10)
y_test's shape :  (486,)


In [4]:

class ContextDataSet(Dataset):
  def __init__(self, target_type, time, tag, before_distance,score):
    ###one hot encoding

    type_converter = {'AT4':0,'MT1':1,'CT1':2,'AD5':3,'FD6':4, 'CE7':5}
    self.target_type = []
    for i in range(len(target_type)):
      self.target_type.insert(i,type_converter[target_type[i]])


    time_converter = {'아침':0,'오전':1,'점심':2,'오후':3,'저녁':4,'밤':5 }
    self.time = []
    for i in range(len(time)):
      self.time.insert(i, time_converter[time[i]])

    tag_converter = {'쇼핑':0,'관광':1,'맛집':2}
    self.tag = []
    for i in range(len(tag)):
      self.tag.insert(i, tag_converter[tag[i]])


    self.before_distance = before_distance
    self.score = score




    self.target_type = nn.functional.one_hot(torch.tensor(self.target_type),6)
    self.time = nn.functional.one_hot(torch.tensor(self.time),6)
    self.tag = nn.functional.one_hot(torch.tensor(self.tag),3)

      


  def __len__(self):
    return len(self.score)
  def __getitem__(self,idx):
    feature = []

    for i in range(len(self.target_type[idx])):
      feature.append(self.target_type[idx][i])
    for i in range(len(self.time[idx])):
      feature.append(self.time[idx][i])
    for i in range(len(self.tag[idx])):
      feature.append(self.tag[idx][i])
    feature.append(self.before_distance[idx]*0.0001)
    

    return feature, self.score[idx]

In [5]:

train_set = ContextDataSet(
    X_train['documents__category_group_code'].values.tolist(),
    X_train['time'].values.tolist(),
    X_train['tag'].values.tolist(),
    X_train['documents__distance'].values.tolist(),
    y_train.to_list()
    )
train_loader = DataLoader(train_set, batch_size = 1, shuffle = True)

test_set = ContextDataSet(
    X_test['documents__category_group_code'].values.tolist(),
    X_test['time'].values.tolist(),
    X_test['tag'].values.tolist(),
    X_test['documents__distance'].values.tolist(),
    y_test.to_list()
    )
test_loader = DataLoader(test_set, batch_size = 1, shuffle = True)

In [6]:
feature,lable = next(iter(train_loader))
print(feature, lable)

[tensor([1]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([0]), tensor([1]), tensor([0]), tensor([0]), tensor([1]), tensor([0]), tensor([0]), tensor([0.1181], dtype=torch.float64)] tensor([7])


In [7]:
### 날씨는 ai안쓰고 거름

In [8]:
class NeuralNetwork(nn.Module):
    def __init__(self):
        super(NeuralNetwork, self).__init__()
        self.layer1 = nn.Sequential(
            nn.Linear(in_features=16, out_features=128, bias = False),
            nn.Sigmoid(),
        )

        self.layer2 = nn.Sequential(
            nn.Linear(in_features=128, out_features=32, bias = False),
        )

        self.layer3 = nn.Sequential(
            nn.Linear(in_features=32, out_features=4, bias = False),
        )

        
        self.layer4 = nn.Sequential(
            nn.Linear(in_features=4, out_features=1, bias = False),
        )


    def forward(self, x):
        x = x.to(device)
        logits = self.layer1(x)
        logits = self.layer2(logits)
        logits = self.layer3(logits)
        logits = self.layer4(logits)
        return logits 

In [9]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(f"Using {device} device")

Using cpu device


In [10]:
model = NeuralNetwork().to(device) # device로 Network 전송
print(model) # 모델 구조 확인

NeuralNetwork(
  (layer1): Sequential(
    (0): Linear(in_features=16, out_features=128, bias=False)
    (1): Sigmoid()
  )
  (layer2): Sequential(
    (0): Linear(in_features=128, out_features=32, bias=False)
  )
  (layer3): Sequential(
    (0): Linear(in_features=32, out_features=4, bias=False)
  )
  (layer4): Sequential(
    (0): Linear(in_features=4, out_features=1, bias=False)
  )
)


In [11]:
# hyperparameter 설정
import torch.optim as optim

#criterion = nn.CrossEntropyLoss() # loss function
criterion = nn.MSELoss() # loss function
optimizer = optim.SGD(model.parameters(), lr=0.001, momentum=0.9) # optimizer

n_epoch = 50  # the number of epochs
n_batch = 10 # the number of batches

In [12]:
for epoch in range(n_epoch):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_loader, 0):

        inputs, labels = data
        
        inputs = torch.tensor([inputs], dtype = torch.float32)
        labels = torch.tensor([labels], dtype = torch.float32)

        inputs = inputs.to(device)
        labels = labels.to(device)

        # optimizer의 파라미터 gradient를 0으로 설정
        optimizer.zero_grad()

        pred = model(inputs).to(device)#예측값 forward

        loss = criterion(pred, labels) #loss값

        loss.backward() #backward
        optimizer.step() #optimize
        #############################################

        # loss 출력
        running_loss += loss.item()
        
        if i % n_batch == 0:    # print every n_batch mini-batches
            print(f'[{epoch + 1}, {i + 1:5d}] loss: {running_loss / n_batch:.3f}')
            running_loss = 0.0

print('Finished Training')

[1,     1] loss: 0.000
[1,    11] loss: 21.484
[1,    21] loss: 10.117
[1,    31] loss: 10.341
[1,    41] loss: 25.725
[1,    51] loss: 12.295
[1,    61] loss: 3.300
[1,    71] loss: 19.057
[1,    81] loss: 10.282

/usr/local/lib/python3.7/dist-packages/torch/nn/modules/loss.py:530: UserWarning: Using a target size (torch.Size([1])) that is different to the input size (torch.Size([1, 1])). This will likely lead to incorrect results due to broadcasting. Please ensure they have the same size.
  return F.mse_loss(input, target, reduction=self.reduction)


스트리밍 출력 내용이 길어서 마지막 5000줄이 삭제되었습니다.
[39,  2571] loss: 1.800
[39,  2581] loss: 2.147
[39,  2591] loss: 4.018
[39,  2601] loss: 1.738
[39,  2611] loss: 0.573
[39,  2621] loss: 1.119
[39,  2631] loss: 1.351
[39,  2641] loss: 1.661
[39,  2651] loss: 2.004
[39,  2661] loss: 2.586
[39,  2671] loss: 0.667
[39,  2681] loss: 1.249
[39,  2691] loss: 0.305
[39,  2701] loss: 0.717
[39,  2711] loss: 3.452
[39,  2721] loss: 1.606
[39,  2731] loss: 3.145
[39,  2741] loss: 1.182
[39,  2751] loss: 0.279
[39,  2761] loss: 2.047
[39,  2771] loss: 0.772
[39,  2781] loss: 1.840
[39,  2791] loss: 0.910
[39,  2801] loss: 1.060
[39,  2811] loss: 0.729
[39,  2821] loss: 1.954
[39,  2831] loss: 1.053
[39,  2841] loss: 1.423
[39,  2851] loss: 0.745
[39,  2861] loss: 1.441
[39,  2871] loss: 1.472
[39,  2881] loss: 0.520
[39,  2891] loss: 0.680
[39,  2901] loss: 1.386
[39,  2911] loss: 2.078
[39,  2921] loss: 1.799
[39,  2931] loss: 1.204
[39,  2941] loss: 1.563
[39,  2951] loss: 1.328
[39,  2961] loss: 0.507
[39,

In [13]:
# 전체 test data에 대한 결과 확인
correct = 0
total = 0

diff_sum = 0

with torch.no_grad(): # 모델을 학습하는 것이 아니므로 gradient 계산을 할 필요가 없음
    for data in test_loader:
        inputs, labels = data

        inputs = torch.tensor([inputs], dtype = torch.float32)
        labels = torch.tensor([labels], dtype = torch.float32)
        inputs = inputs.to(device)
        labels = labels.to(device)

        outputs = model(inputs)

        #_, predicted = torch.max(outputs.data, 1)

        predicted = outputs
        print(predicted.item())
        print(labels.item())
        diff = predicted.item() - labels.item()
        diff_sum+= diff

        total += labels.size(0)


        correct += (predicted == labels.to(device)).sum().item()

print(f'Accuracy of the network on the test images: {100 * correct // total} %')
diff_avg = diff_sum / total
print(diff_avg)

4.565778732299805
6.0
0.2112380415201187
0.0
2.0665669441223145
2.0
2.270786762237549
1.0
0.970781683921814
0.0
12.474265098571777
10.0
9.218910217285156
10.0
1.303917407989502
1.0
7.823185443878174
8.0
2.4036431312561035
3.0
4.0075907707214355
3.0
3.9620070457458496
4.0
2.1564507484436035
3.0
1.1238291263580322
1.0
6.153095245361328
7.0
6.310865879058838
7.0
3.682133197784424
4.0
0.5976304411888123
0.0
6.192580699920654
5.0
0.5684528946876526
0.0
4.794363975524902
4.0
0.02049075998365879
0.0
2.304534435272217
1.0
6.33408260345459
6.0
3.7225968837738037
5.0
2.91909122467041
3.0
1.2310960292816162
1.0
5.463627815246582
7.0
2.3705315589904785
2.0
1.6413500308990479
2.0
0.4831995964050293
0.0
0.04188182204961777
0.0
0.052022699266672134
0.0
5.614943027496338
5.0
2.056346893310547
4.0
6.496282577514648
6.0
5.112741470336914
5.0
0.4666992425918579
0.0
1.6565607786178589
4.0
0.4429479241371155
0.0
0.02049075998365879
0.0
0.11701543629169464
0.0
1.189342737197876
1.0
0.005367540754377842
0.0


In [14]:
!curl -v -X GET "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=AT4&radius=10000&y=33.4889179032603&x=126.498229141199" -H "Authorization: KakaoAK 9a12f840d08a0b698f040ffc93f18211" > AT4
!curl -v -X GET "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=MT1&radius=10000&y=33.4889179032603&x=126.498229141199" -H "Authorization: KakaoAK 9a12f840d08a0b698f040ffc93f18211" > MT1
!curl -v -X GET "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=CT1&radius=10000&y=33.4889179032603&x=126.498229141199" -H "Authorization: KakaoAK 9a12f840d08a0b698f040ffc93f18211" > CT1
!curl -v -X GET "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=AD5&radius=10000&y=33.4889179032603&x=126.498229141199" -H "Authorization: KakaoAK 9a12f840d08a0b698f040ffc93f18211" > AD5
!curl -v -X GET "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=FD6&radius=10000&y=33.4889179032603&x=126.498229141199" -H "Authorization: KakaoAK 9a12f840d08a0b698f040ffc93f18211" > FD6
!curl -v -X GET "https://dapi.kakao.com/v2/local/search/category.json?category_group_code=CE7&radius=10000&y=33.4889179032603&x=126.498229141199" -H "Authorization: KakaoAK 9a12f840d08a0b698f040ffc93f18211" > CE7

Note: Unnecessary use of -X or --request, GET is already inferred.
  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0*   Trying 211.249.220.147...
* TCP_NODELAY set
* Connected to dapi.kakao.com (211.249.220.147) port 443 (#0)
  0     0    0     0    0     0      0      0 --:--:-- --:--:-- --:--:--     0* ALPN, offering h2
* ALPN, offering http/1.1
* successfully set certificate verify locations:
*   CAfile: /etc/ssl/certs/ca-certificates.crt
  CApath: /etc/ssl/certs
} [5 bytes data]
* TLSv1.3 (OUT), TLS handshake, Client hello (1):
} [512 bytes data]
* TLSv1.3 (IN), TLS handshake, Server hello (2):
{ [106 bytes data]
* TLSv1.2 (IN), TLS handshake, Certificate (11):
{ [3727 bytes data]
* TLSv1.2 (IN), TLS handshake, Server key exchange (12):
{ [300 bytes data]
* TLSv1.2 (IN), TLS handshake, Server finis

In [15]:
AT4 = open('AT4')
jsonString = json.load(AT4)
print(jsonString.get("documents")[0])

{'address_name': '제주특별자치도 제주시 연동 281-20', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 동물원 > 실내동물원', 'distance': '563', 'id': '1862508988', 'phone': '064-711-1145', 'place_name': '캐니언파크 제주점', 'place_url': 'http://place.map.kakao.com/1862508988', 'road_address_name': '제주특별자치도 제주시 삼무로 51', 'x': '126.49236793947', 'y': '33.490237404658'}


In [16]:
place_data = []


AT4 = open('AT4')
jsonString = json.load(AT4)
for i in range(len(jsonString.get("documents"))):
  place_data.append(jsonString.get("documents")[i])

MT1 = open('MT1')
jsonString = json.load(MT1)
for i in range(len(jsonString.get("documents"))):
  place_data.append(jsonString.get("documents")[i])

CT1 = open('CT1')
jsonString = json.load(CT1)
for i in range(len(jsonString.get("documents"))):
  place_data.append(jsonString.get("documents")[i])

AD5 = open('AD5')
jsonString = json.load(AD5)
for i in range(len(jsonString.get("documents"))):
  place_data.append(jsonString.get("documents")[i])

FD6 = open('FD6')
jsonString = json.load(FD6)
for i in range(len(jsonString.get("documents"))):
  place_data.append(jsonString.get("documents")[i])
  
CE7 = open('CE7')
jsonString = json.load(CE7)
for i in range(len(jsonString.get("documents"))):
  place_data.append(jsonString.get("documents")[i])



In [17]:
print(place_data)

[{'address_name': '제주특별자치도 제주시 연동 281-20', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 동물원 > 실내동물원', 'distance': '563', 'id': '1862508988', 'phone': '064-711-1145', 'place_name': '캐니언파크 제주점', 'place_url': 'http://place.map.kakao.com/1862508988', 'road_address_name': '제주특별자치도 제주시 삼무로 51', 'x': '126.49236793947', 'y': '33.490237404658'}, {'address_name': '제주특별자치도 제주시 연동 273', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 테마거리', 'distance': '773', 'id': '21811778', 'phone': '', 'place_name': '누웨마루거리', 'place_url': 'http://place.map.kakao.com/21811778', 'road_address_name': '', 'x': '126.490657044139', 'y': '33.48601183492031'}, {'address_name': '제주특별자치도 제주시 오라이동 1758', 'category_group_code': 'AT4', 'category_group_name': '관광명소', 'category_name': '여행 > 관광,명소 > 테마파크', 'distance': '1263', 'id': '965883722', 'phone': '070-4647-0559', 'place_name': '바운스슈퍼파크 제주센터', 'place_url': 'http://place.map.kakao.

In [20]:
with torch.no_grad(): # 모델을 학습하는 것이 아니므로 gradient 계산을 할 필요가 없음
    for i in range(len(place_data)):

        type_converter = {'AT4':0,'MT1':1,'CT1':2,'AD5':3,'FD6':4, 'CE7':5}
        group_code_raw = place_data[i].get("category_group_code")
        group_code = type_converter[group_code_raw]
        


        time_converter = {'아침':0,'오전':1,'점심':2,'오후':3,'저녁':4,'밤':5 }
        time_raw = '오후'
        time = time_converter[time_raw]


        tag_converter = {'쇼핑':0,'관광':1,'맛집':2}
        tag_raw = '맛집'
        tag = tag_converter[tag_raw]



        distance = place_data[i].get("distance")



        group_code = nn.functional.one_hot(torch.tensor(group_code),6)
        time = nn.functional.one_hot(torch.tensor(time),6)
        tag = nn.functional.one_hot(torch.tensor(tag),3)
            
        feature = []

        for j in range(len(group_code)):
          feature.append(group_code[j])
        for j in range(len(time)):
          feature.append(time[j])
        for j in range(len(tag)):
          feature.append(tag[j])
        feature.append(float(distance)*0.0001)

        
        inputs = feature
        inputs = torch.tensor([inputs], dtype = torch.float32)
        inputs = inputs.to(device)

        predicted = model(inputs)

        place_data[i]["score"] = predicted.item()
        group_code_name = place_data[i].get("category_group_name")
        #print(i, group_code_name,time_raw,distance, place_data[i]["score"])

ret = sorted(place_data, key=lambda place_data: place_data["score"], reverse=True)
for i in range(10):
  print(ret[i])

{'address_name': '제주특별자치도 제주시 연동 322-10', 'category_group_code': 'CE7', 'category_group_name': '카페', 'category_name': '음식점 > 카페 > 커피전문점', 'distance': '207', 'id': '27563112', 'phone': '070-7807-5605', 'place_name': '베이스노트', 'place_url': 'http://place.map.kakao.com/27563112', 'road_address_name': '제주특별자치도 제주시 선덕로5길 21', 'x': '126.499940427066', 'y': '33.4877202928652', 'score': 9.789588928222656}
{'address_name': '제주특별자치도 제주시 연동 311-17', 'category_group_code': 'CE7', 'category_group_name': '카페', 'category_name': '음식점 > 카페', 'distance': '257', 'id': '1230584692', 'phone': '070-8878-3739', 'place_name': '청춘다방', 'place_url': 'http://place.map.kakao.com/1230584692', 'road_address_name': '제주특별자치도 제주시 귀아랑길 27', 'x': '126.4975490127217', 'y': '33.49116912810348', 'score': 9.33590316772461}
{'address_name': '제주특별자치도 제주시 연동 322-2', 'category_group_code': 'CT1', 'category_group_name': '문화시설', 'category_name': '문화,예술 > 문화시설 > 전시관', 'distance': '284', 'id': '2080157074', 'phone': '', 'place_name': 